# Relative clauses
## Author: Kyle Rawlins

This notebook goes through the analysis of relative clauses in Heim & Kratzer (1998), "Semantics in generative grammar".

In [ ]:
reload_lamb()

In [ ]:
composition_system = lang.hk3_system.copy()
lang.set_system(composition_system)

In [ ]:
%%lamb reset
||gray|| = lambda x_e : Gray(x)
||cat|| = lambda x_e : Cat(x)
||bill|| = Bill_e
||likes|| = lambda x_e : lambda y_e : Likes(y,x)

In [ ]:
gray * cat

Machinery for variable binding.  This takes two parts:

 * specialized Item classes for the binder and the trace.
 * code to actually perform predicate abstraction.  This version uses the meta-language assignment function.

In [ ]:
class Binder(lang.Item):
    def __init__(self, index):
        lang.Item.__init__(self, "%i" % index, None, index=index)

class Trace(lang.Item):
    def __init__(self, index, typ=None):
        if typ is None:
            typ = types.type_e
        if index > 0:
            name = "t%i" % index
        else:
            name = "t"
        # Item constructor will set self.index
        lang.Item.__init__(self, name, meta.TypedTerm(name, typ), index=index)  

In [ ]:
def tree_pa_fun(t, assignment=None):
    binder = t[0]
    if (binder.content is not None) or not binder.name.strip().isnumeric():
        raise types.TypeMismatch(t, None, "Predicate Abstraction")
    index = int(binder.name.strip())
    vname = "t%i" % index
    outer_vname = t[1].content.find_safe_variable()
    new_a = lang.Assignment(assignment)
    new_a.update({vname: lang.te("%s_e" % outer_vname)})
    f = meta.LFun(types.type_e, t[1].content.under_assignment(new_a), varname=outer_vname)
    return lang.BinaryComposite(t[0], t[1], f)

pa_op = lang.TreeCompositionOp("PA", tree_pa_fun, allow_none=True)
lang.get_system().add_rule(pa_op)

Now let's use these things.  Binders and traces can easily be constructed around numeric indices, and used anywhere you would use a regular lexical item:

In [ ]:
b = Binder(5)
t = Trace(5)
t

In [ ]:
b * t

In [ ]:
bill * (likes * t)

A simple relative clause like "that Bill likes" can be represented as follows:

In [ ]:
r = b * (bill * (likes * t))
r.paths(derivations=True)

This would compose with a NP via Predicate Modification.

In [ ]:
r = gray * (cat * (b * (bill * (likes * t))))
r

In [ ]:
r.paths()

Now for a very complicated example.  This is one of the more challenging examples from Heim and Kratzer, the such-that relative "the man such that Mary reviewed a book that he wrote".

In [ ]:
%%lamb
||mary|| = Mary_e
||man|| = L x_e : Man(x)
||book|| = L x_e : Book(x)
||wrote|| = L x_e : L y_e : Wrote(y,x)
||reviewed|| = L x_e : L y_e : Reviewed(y,x)
||that|| = L p_t : p
||such|| = L p_t : p
||a|| = L f_<e,t> : L g_<e,<e,t>> : L y_e : Exists x_e : f(x) & g(x)(y)

In [ ]:
r = man * (Binder(2) * (such * (that * (mary * (reviewed * (a * (book * (Binder(1) * (Trace(2) * (wrote * Trace(1)))))))))))
r

In [ ]:
r.paths()